In [1]:
import torch
import json
import os
import random
import numpy as np
import torch
from sklearn import preprocessing
import torch.multiprocessing as mp
import torch
from torch.utils.data import DataLoader
import copy
from torch import nn
%matplotlib inline
import matplotlib.pyplot as plt
from dnaDataloader import expermentDataloader
from dnaDataloader import addData
from scipy import stats as st
from dnaModelUtil import train


In [2]:
try:
   mp.set_start_method('spawn')
except RuntimeError:
   pass # throws error if run twice without resetting the kernal, if its already set we dont care that this errors
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")
# num_workers = 25
batch_size = 25
device

device(type='cuda', index=1)

## Eval with small traing sets
The goal here is to eval the model while training on small sets of data. That is how well can MLP learn from 100-200 examples of single-molecule experiments.
A verity of samples have been provided anf preprocessed (converted from excel to csv and had empty frames added) to the folder <pre>/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted</pre>

In [3]:
folders = [d[0] for d in os.walk("/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted")][1:] # remove first one is it is "/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted"
len(folders)

42

In [4]:
datasets = []
featIn = 0
for d in folders:
    data = expermentDataloader(
        f"{d}/index.csv",
        f"{d}", 
    )
    rawData = [d for d in data]
    featIn = len(rawData[0][0])
    trainValidData = []
    testData = []
    addData(testData, trainValidData, rawData, rhsSize=300)


    np.random.shuffle(trainValidData)
    trainData = []
    validData = []
    addData(trainData, validData, trainValidData, rhsSize=int(len(trainValidData)*(1/3)))

    datasets.append({"name": f"{os.path.basename(d)}", 
                     "train":DataLoader(trainData, batch_size=batch_size, shuffle=True) , 
                     "valid":DataLoader(validData, batch_size=batch_size, shuffle=True) , 
                     "test":DataLoader(testData, batch_size=len(testData), shuffle=True) ,
                     "featIn": featIn,
                     "model": {}}
                    )

In [5]:
print(f"datasets:")
for d in datasets:
    print(d)
print(f"{len(datasets)}")
print(f"featIn: {featIn}")

datasets:
{'name': '1800_nM_AR_2100', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f8462760a60>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f845b840910>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f845b679240>, 'featIn': 42000, 'model': {}}
{'name': '800_nM_AR_1200', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f85c03ce350>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f845b63a830>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f845b66f220>, 'featIn': 24000, 'model': {}}
{'name': '100_nM_AR_2100', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f85c03ce6b0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f845b672140>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f8454cebc70>, 'featIn': 42000, 'model': {}}
{'name': '400_nM_AR_2400', 'train': <torch.utils.data.dataloader.DataLoader object at 0x7f845b670190>, 'valid': <torch.utils.data.dataloa

In [6]:
for d in datasets:
    print(f"-- {d['name']} --")
    print(f"train: {len(d['train'])}")
    print(f"valid: {len(d['valid'])}")
    print(f"test : {len(d['test'])}")

-- 1800_nM_AR_2100 --
train: 8
valid: 4
test : 1
-- 800_nM_AR_1200 --
train: 8
valid: 4
test : 1
-- 100_nM_AR_2100 --
train: 8
valid: 4
test : 1
-- 400_nM_AR_2400 --
train: 8
valid: 4
test : 1
-- 400_nM_AR_1800 --
train: 8
valid: 4
test : 1
-- 1200_nM_AR_1200 --
train: 8
valid: 4
test : 1
-- 400_nM_AR_900 --
train: 8
valid: 4
test : 1
-- 1200_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 100_nM_AR_900 --
train: 8
valid: 4
test : 1
-- 1800_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 1800_nM_AR_2400 --
train: 8
valid: 4
test : 1
-- 1200_nM_AR_1800 --
train: 8
valid: 4
test : 1
-- 1800_nM_AR_1800 --
train: 8
valid: 4
test : 1
-- 50_nM_AR_1200 --
train: 8
valid: 4
test : 1
-- 100_nM_AR_1200 --
train: 8
valid: 4
test : 1
-- 50_nM_AR_2400 --
train: 8
valid: 4
test : 1
-- 100_nM_AR_600 --
train: 8
valid: 4
test : 1
-- 1800_nM_AR_1200 --
train: 8
valid: 4
test : 1
-- 400_nM_AR_1500 --
train: 8
valid: 4
test : 1
-- 800_nM_AR_900 --
train: 8
valid: 4
test : 1
-- 50_nM_AR_900 --
train: 8
valid: 4
tes

In [7]:
datasets[0]

{'name': '1800_nM_AR_2100',
 'train': <torch.utils.data.dataloader.DataLoader at 0x7f8462760a60>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x7f845b840910>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f845b679240>,
 'featIn': 42000,
 'model': {}}

In [8]:
manager = mp.Manager()
return_dict = manager.dict()

In [9]:
datasets = [datasets.pop(8)] + datasets[:8] + datasets[9:]
datasets

[{'name': '100_nM_AR_900',
  'train': <torch.utils.data.dataloader.DataLoader at 0x7f8454bbfc70>,
  'valid': <torch.utils.data.dataloader.DataLoader at 0x7f8454bc8430>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f8454b73c70>,
  'featIn': 18000,
  'model': {}},
 {'name': '1800_nM_AR_2100',
  'train': <torch.utils.data.dataloader.DataLoader at 0x7f8462760a60>,
  'valid': <torch.utils.data.dataloader.DataLoader at 0x7f845b840910>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f845b679240>,
  'featIn': 42000,
  'model': {}},
 {'name': '800_nM_AR_1200',
  'train': <torch.utils.data.dataloader.DataLoader at 0x7f85c03ce350>,
  'valid': <torch.utils.data.dataloader.DataLoader at 0x7f845b63a830>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f845b66f220>,
  'featIn': 24000,
  'model': {}},
 {'name': '100_nM_AR_2100',
  'train': <torch.utils.data.dataloader.DataLoader at 0x7f85c03ce6b0>,
  'valid': <torch.utils.data.dataloader.DataLoader at 0x7f845b672140>,
  'te

In [10]:
processes = []
devices = [torch.device("cuda:0"),torch.device("cuda:1"),torch.device("cuda:2"),torch.device("cuda:3")]
epochs = 5000
error_margin = 20
for d in datasets:
    try:
        train(d["train"], d["valid"], d["name"], d["featIn"], return_dict, epochs, error_margin, devices[0], capacity=int(d["name"].split('_')[-1]) )
    except:
        print("skipping " + d["name"])
    # processes.append(mp.Process(target=train, args=(d["train"], d["valid"], d["name"], d["featIn"], return_dict, epochs, error_margin, devices[0])))
    # devices.append(devices.pop(0))
    
# processes

training 100_nM_AR_900 on cuda:0...
training 1800_nM_AR_2100 on cuda:0...
training 800_nM_AR_1200 on cuda:0...
training 100_nM_AR_2100 on cuda:0...
training 400_nM_AR_2400 on cuda:0...
training 400_nM_AR_1800 on cuda:0...


In [ ]:
print(return_dict)

{'100_nM_AR_900': {'path': './Models/smallTrain/100_nM_AR_900.pt', 'acc': 0.45000000000000007}, '1800_nM_AR_2100': {'path': './Models/smallTrain/1800_nM_AR_2100.pt', 'acc': 0.04}, '800_nM_AR_1200': {'path': './Models/smallTrain/800_nM_AR_1200.pt', 'acc': 0.08}, '100_nM_AR_2100': {'path': './Models/smallTrain/100_nM_AR_2100.pt', 'acc': 0.0}, '400_nM_AR_2400': {'path': './Models/smallTrain/400_nM_AR_2400.pt', 'acc': 0.01}, '400_nM_AR_1800': {'path': './Models/smallTrain/400_nM_AR_1800.pt', 'acc': 0.01}, '1200_nM_AR_1200': {'path': './Models/smallTrain/1200_nM_AR_1200.pt', 'acc': 0.24000000000000002}, '400_nM_AR_900': {'path': './Models/smallTrain/400_nM_AR_900.pt', 'acc': 0.22000000000000003}, '1800_nM_AR_2400': {'path': './Models/smallTrain/1800_nM_AR_2400.pt', 'acc': 0.03}, '1200_nM_AR_1800': {'path': './Models/smallTrain/1200_nM_AR_1800.pt', 'acc': 0.17}, '1800_nM_AR_1800': {'path': './Models/smallTrain/1800_nM_AR_1800.pt', 'acc': 0.060000000000000005}, '50_nM_AR_1200': {'path': './Mo

In [ ]:
# processesList = list(range(len(processes)))

# while processesList:
#     run = processesList[:4]
#     processesList = processesList[4:]
#     for i in run:
#         processes[i].start()
#     for i in run:
#         processes[i].join()
#         processes[i].terminate()
# print(return_dict)

In [ ]:
results = dict(return_dict)
with open("./Models/smallTrain/results.json", 'w') as file:
    json_object = json.dumps(results, indent=4)
    file.write(json_object)